In [2]:
import os, shutil, random
from pathlib import Path

# ==== EDIT THESE TO YOUR FOLDERS ====
FOLD1_DIR = Path(r"D:\psm2\Fold1")     # has train/val/test for monkeypox & others
NORMAL_DIR = Path(r"D:\psm2\Normal")   # a flat folder of Normal images (not split yet)
OUTPUT_DIR = Path(r"D:\psm2\MediSkinDataset")  # final merged dataset
CLASS_NAME = "normal"                  # folder name for the class

# ---------- Helpers ----------
IMG_EXT = {".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp"}

def list_images(root: Path):
    return [p for p in root.rglob("*") if p.is_file() and p.suffix.lower() in IMG_EXT]

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def copy_all(files, dst_dir: Path, prefix="normal"):
    ensure_dir(dst_dir)
    for i, src in enumerate(files, 1):
        dst = dst_dir / f"{prefix}_{i:05d}{src.suffix.lower()}"
        shutil.copy2(src, dst)

# ---------- 1) Read Fold1 ratios ----------
# Count total images across train/val/test (use one existing class to infer ratios)
def split_ratio_from_fold1():
    # pick the first class we find under train to anchor ratios
    train_classes = [d for d in (FOLD1_DIR/"train").iterdir() if d.is_dir()]
    if not train_classes:
        raise RuntimeError("No classes found under Fold1/train")

    anchor_class = train_classes[0].name
    n_train = len(list_images(FOLD1_DIR/"train"/anchor_class))
    n_val   = len(list_images(FOLD1_DIR/"val"/anchor_class)) if (FOLD1_DIR/"val").exists() else 0
    n_test  = len(list_images(FOLD1_DIR/"test"/anchor_class))

    total = n_train + n_val + n_test
    if total == 0:
        raise RuntimeError("Fold1 appears empty.")

    r_train = n_train / total
    r_val   = n_val   / total
    r_test  = n_test  / total
    return r_train, r_val, r_test

r_train, r_val, r_test = split_ratio_from_fold1()
print(f"Using ratios from Fold1 → train:{r_train:.3f}  val:{r_val:.3f}  test:{r_test:.3f}")

# ---------- 2) Collect Normal images and split ----------
normal_imgs = [p for p in NORMAL_DIR.iterdir() if p.is_file() and p.suffix.lower() in IMG_EXT]
if not normal_imgs:
    raise RuntimeError("No images found in the Normal folder.")

random.seed(42)
random.shuffle(normal_imgs)

N = len(normal_imgs)
n_train = int(round(r_train * N))
n_val   = int(round(r_val   * N))
# put the remainder in test to ensure totals add up
n_test  = N - n_train - n_val

train_files = normal_imgs[:n_train]
val_files   = normal_imgs[n_train:n_train+n_val]
test_files  = normal_imgs[n_train+n_val:]

print(f"Normal split → train:{len(train_files)}  val:{len(val_files)}  test:{len(test_files)}  (total {N})")

# ---------- 3) Build output structure ----------
# First, mirror Fold1 into OUTPUT_DIR
for split in ["train","val","test"]:
    src = FOLD1_DIR/split
    if not src.exists(): 
        continue
    for cls_dir in [d for d in src.iterdir() if d.is_dir()]:
        dst = OUTPUT_DIR/split/cls_dir.name
        ensure_dir(dst)
        # copy existing monkeypox/others (keep filenames)
        for img in list_images(cls_dir):
            shutil.copy2(img, dst/img.name)

# Now add Normal into OUTPUT_DIR
copy_all(train_files, OUTPUT_DIR/"train"/CLASS_NAME, prefix="normal")
if r_val > 0:
    copy_all(val_files,   OUTPUT_DIR/"val"/CLASS_NAME,   prefix="normal")
copy_all(test_files,  OUTPUT_DIR/"test"/CLASS_NAME,  prefix="normal")

print("✅ Done. Final dataset at:", OUTPUT_DIR)
# You should now have:
# MediSkinDataset/
#   train/  monkeypox/ others/ normal/
#   val/    monkeypox/ others/ normal/   (only if Fold1 had val)
#   test/   monkeypox/ others/ normal/


Using ratios from Fold1 → train:0.839  val:0.144  test:0.017
Normal split → train:341  val:59  test:7  (total 407)
✅ Done. Final dataset at: D:\psm2\MediSkinDataset


In [3]:
# train_disease_mnv2.py
import os, json, tensorflow as tf
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras import layers, models, callbacks, optimizers

DATA = r"D:\psm2\MediSkinDataset"
OUT  = r"D:\psm2\models"; os.makedirs(OUT, exist_ok=True)
IMG=(224,224); BATCH=32

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, rotation_range=12, width_shift_range=0.05, height_shift_range=0.05,
    zoom_range=0.15, horizontal_flip=True)
val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory(f"{DATA}\\train", target_size=IMG, batch_size=BATCH, class_mode="categorical")
val_ds   = val_gen.flow_from_directory(f"{DATA}\\val",   target_size=IMG, batch_size=BATCH, class_mode="categorical")

with open(os.path.join(OUT,"disease_class_indices.json"),"w") as f:
    json.dump(train_ds.class_indices, f, indent=2)

base = mobilenet_v2.MobileNetV2(weights="imagenet", include_top=False, input_shape=IMG+(3,))
base.trainable=False
model = models.Sequential([ base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(train_ds.num_classes, activation="softmax") ])

model.compile(optimizer=optimizers.Adam(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])
ckp = callbacks.ModelCheckpoint(r"D:\psm2\models\disease_mnv2.h5", save_best_only=True, monitor="val_accuracy")
es  = callbacks.EarlyStopping(patience=6, restore_best_weights=True)
rlr = callbacks.ReduceLROnPlateau(patience=3, factor=0.3)

model.fit(train_ds, epochs=25, validation_data=val_ds, callbacks=[ckp, es, rlr])

# optional fine-tune tail
base.trainable=True
for layer in base.layers[:-20]: layer.trainable=False
model.compile(optimizer=optimizers.Adam(1e-4), loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=[ckp, es, rlr])
print("saved model to D:\\psm2\\models\\disease_mnv2.h5")


Found 2483 images belonging to 3 classes.
Found 479 images belonging to 3 classes.


C:\Users\Fazreen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6068 - loss: 1.0126

C:\Users\Fazreen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


78/78 ━━━━━━━━━━━━━━━━━━━━ 212s 3s/step - accuracy: 0.6079 - loss: 1.0093 - val_accuracy: 0.7578 - val_loss: 0.4879 - learning_rate: 0.0010
Epoch 2/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8158 - loss: 0.4343

78/78 ━━━━━━━━━━━━━━━━━━━━ 162s 2s/step - accuracy: 0.8158 - loss: 0.4341 - val_accuracy: 0.7787 - val_loss: 0.4614 - learning_rate: 0.0010
Epoch 3/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - accuracy: 0.8416 - loss: 0.3812 - val_accuracy: 0.7557 - val_loss: 0.4654 - learning_rate: 0.0010
Epoch 4/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 152s 2s/step - accuracy: 0.8605 - loss: 0.3396 - val_accuracy: 0.7474 - val_loss: 0.4920 - learning_rate: 0.0010
Epoch 5/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - accuracy: 0.8739 - loss: 0.2988 - val_accuracy: 0.7599 - val_loss: 0.4951 - learning_rate: 0.0010
Epoch 6/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8821 - loss: 0.2635

78/78 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.8821 - loss: 0.2633 - val_accuracy: 0.7954 - val_loss: 0.4230 - learning_rate: 3.0000e-04
Epoch 7/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8782 - loss: 0.2562

78/78 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.8784 - loss: 0.2560 - val_accuracy: 0.7975 - val_loss: 0.4365 - learning_rate: 3.0000e-04
Epoch 8/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9067 - loss: 0.2235

78/78 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.9067 - loss: 0.2235 - val_accuracy: 0.8079 - val_loss: 0.4292 - learning_rate: 3.0000e-04
Epoch 9/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - accuracy: 0.9114 - loss: 0.2097 - val_accuracy: 0.7704 - val_loss: 0.4695 - learning_rate: 3.0000e-04
Epoch 10/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.9010 - loss: 0.2028 - val_accuracy: 0.7808 - val_loss: 0.4528 - learning_rate: 9.0000e-05
Epoch 11/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.9284 - loss: 0.1755 - val_accuracy: 0.7808 - val_loss: 0.4547 - learning_rate: 9.0000e-05
Epoch 12/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 134s 2s/step - accuracy: 0.9171 - loss: 0.1982 - val_accuracy: 0.7641 - val_loss: 0.4784 - learning_rate: 9.0000e-05
Epoch 1/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.7374 - loss: 0.6494 - val_accuracy: 0.6639 - val_loss: 1.5658 - learning_rate: 1.0000e-04
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.8785 - loss: 

In [4]:
# train_disease_resnet50.py
import os, json, tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras import layers, models, callbacks, optimizers

DATA = r"D:\psm2\MediSkinDataset"
OUT  = r"D:\psm2\models"; os.makedirs(OUT, exist_ok=True)
IMG=(224,224); BATCH=32

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, rotation_range=12, width_shift_range=0.05,
    height_shift_range=0.05, zoom_range=0.15, horizontal_flip=True)
val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory(f"{DATA}\\train", target_size=IMG, batch_size=BATCH, class_mode="categorical")
val_ds   = val_gen.flow_from_directory(f"{DATA}\\val",   target_size=IMG, batch_size=BATCH, class_mode="categorical")

with open(os.path.join(OUT,"disease_class_indices.json"),"w") as f:
    json.dump(train_ds.class_indices, f, indent=2)

base = resnet50.ResNet50(weights="imagenet", include_top=False, input_shape=IMG+(3,))
base.trainable=False

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(train_ds.num_classes, activation="softmax")
])

model.compile(optimizer=optimizers.Adam(1e-3),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

ckp = callbacks.ModelCheckpoint(r"D:\psm2\models\disease_resnet50.h5", save_best_only=True, monitor="val_accuracy")
es  = callbacks.EarlyStopping(patience=6, restore_best_weights=True)
rlr = callbacks.ReduceLROnPlateau(patience=3, factor=0.3)

model.fit(train_ds, epochs=25, validation_data=val_ds, callbacks=[ckp, es, rlr])

# fine-tune last block
base.trainable=True
for layer in base.layers[:-30]:
    layer.trainable=False

model.compile(optimizer=optimizers.Adam(1e-4), loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=[ckp, es, rlr])
print("✅ Saved ResNet50 model")


Found 2483 images belonging to 3 classes.
Found 479 images belonging to 3 classes.
Epoch 1/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4129 - loss: 1.1872

78/78 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.4132 - loss: 1.1858 - val_accuracy: 0.5511 - val_loss: 0.8617 - learning_rate: 0.0010
Epoch 2/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4667 - loss: 0.9639

78/78 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.4669 - loss: 0.9637 - val_accuracy: 0.6973 - val_loss: 0.8142 - learning_rate: 0.0010
Epoch 3/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.4945 - loss: 0.9122 - val_accuracy: 0.6868 - val_loss: 0.7857 - learning_rate: 0.0010
Epoch 4/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.5205 - loss: 0.8991 - val_accuracy: 0.6430 - val_loss: 0.7565 - learning_rate: 0.0010
Epoch 5/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.5202 - loss: 0.8777 - val_accuracy: 0.6451 - val_loss: 0.7748 - learning_rate: 0.0010
Epoch 6/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.5374 - loss: 0.8755 - val_accuracy: 0.6618 - val_loss: 0.7657 - learning_rate: 0.0010
Epoch 7/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.5416 - loss: 0.8697 - val_accuracy: 0.6555 - val_loss: 0.7415 - learning_rate: 0.0010
Epoch 8/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.5542 - loss: 0.8777 - val_accuracy: 0.66

78/78 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.6660 - loss: 0.7076 - val_accuracy: 0.7223 - val_loss: 0.6404 - learning_rate: 3.0000e-05
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6667 - loss: 0.6741

78/78 ━━━━━━━━━━━━━━━━━━━━ 199s 3s/step - accuracy: 0.6668 - loss: 0.6740 - val_accuracy: 0.7578 - val_loss: 0.6061 - learning_rate: 3.0000e-05
Epoch 8/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 0.6892 - loss: 0.6517 - val_accuracy: 0.5031 - val_loss: 0.8659 - learning_rate: 3.0000e-05
Epoch 9/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 204s 3s/step - accuracy: 0.6766 - loss: 0.6788 - val_accuracy: 0.4175 - val_loss: 1.7385 - learning_rate: 3.0000e-05
Epoch 10/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.7054 - loss: 0.6296 - val_accuracy: 0.4990 - val_loss: 1.1733 - learning_rate: 3.0000e-05
✅ Saved ResNet50 model


In [5]:
# train_disease_inceptionv3.py
import os, json, tensorflow as tf
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import layers, models, callbacks, optimizers

DATA = r"D:\psm2\MediSkinDataset"
OUT  = r"D:\psm2\models"; os.makedirs(OUT, exist_ok=True)
IMG=(224,224); BATCH=32

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, rotation_range=12, width_shift_range=0.05,
    height_shift_range=0.05, zoom_range=0.15, horizontal_flip=True)
val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory(f"{DATA}\\train", target_size=IMG, batch_size=BATCH, class_mode="categorical")
val_ds   = val_gen.flow_from_directory(f"{DATA}\\val",   target_size=IMG, batch_size=BATCH, class_mode="categorical")

with open(os.path.join(OUT,"disease_class_indices.json"),"w") as f:
    json.dump(train_ds.class_indices, f, indent=2)

base = inception_v3.InceptionV3(weights="imagenet", include_top=False, input_shape=IMG+(3,))
base.trainable=False

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(train_ds.num_classes, activation="softmax")
])

model.compile(optimizer=optimizers.Adam(1e-3),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

ckp = callbacks.ModelCheckpoint(r"D:\psm2\models\disease_inceptionv3.h5", save_best_only=True, monitor="val_accuracy")
es  = callbacks.EarlyStopping(patience=6, restore_best_weights=True)
rlr = callbacks.ReduceLROnPlateau(patience=3, factor=0.3)

model.fit(train_ds, epochs=25, validation_data=val_ds, callbacks=[ckp, es, rlr])

# fine-tune tail
base.trainable=True
for layer in base.layers[:-30]:
    layer.trainable=False

model.compile(optimizer=optimizers.Adam(1e-4), loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=[ckp, es, rlr])
print("✅ Saved InceptionV3 model")


Found 2483 images belonging to 3 classes.
Found 479 images belonging to 3 classes.
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 857ms/step - accuracy: 0.6649 - loss: 0.8359

78/78 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.6657 - loss: 0.8336 - val_accuracy: 0.7996 - val_loss: 0.4685 - learning_rate: 0.0010
Epoch 2/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.8062 - loss: 0.4613 - val_accuracy: 0.7537 - val_loss: 0.5043 - learning_rate: 0.0010
Epoch 3/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.8500 - loss: 0.3684 - val_accuracy: 0.7871 - val_loss: 0.4977 - learning_rate: 0.0010
Epoch 4/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - accuracy: 0.8630 - loss: 0.3540 - val_accuracy: 0.7766 - val_loss: 0.4661 - learning_rate: 0.0010
Epoch 5/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.8727 - loss: 0.3214 - val_accuracy: 0.7557 - val_loss: 0.4794 - learning_rate: 0.0010
Epoch 6/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.8840 - loss: 0.2889 - val_accuracy: 0.7787 - val_loss: 0.4810 - learning_rate: 0.0010
Epoch 7/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8974 - loss: 0.2511

78/78 ━━━━━━━━━━━━━━━━━━━━ 98s 1s/step - accuracy: 0.8974 - loss: 0.2512 - val_accuracy: 0.8038 - val_loss: 0.4269 - learning_rate: 0.0010
Epoch 8/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.8945 - loss: 0.2566 - val_accuracy: 0.7557 - val_loss: 0.4956 - learning_rate: 0.0010
Epoch 9/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 92s 1s/step - accuracy: 0.9000 - loss: 0.2168 - val_accuracy: 0.7390 - val_loss: 0.5347 - learning_rate: 0.0010
Epoch 10/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.9187 - loss: 0.2143 - val_accuracy: 0.7704 - val_loss: 0.5442 - learning_rate: 0.0010
Epoch 11/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 92s 1s/step - accuracy: 0.9202 - loss: 0.2045 - val_accuracy: 0.7829 - val_loss: 0.4736 - learning_rate: 3.0000e-04
Epoch 12/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - accuracy: 0.9371 - loss: 0.1628 - val_accuracy: 0.7432 - val_loss: 0.5048 - learning_rate: 3.0000e-04
Epoch 13/25
78/78 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.9226 - loss: 0.1819 - val_accuracy:

In [7]:
pip install scikit-learn


   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
    --------------------------------------- 0.1/8.9 MB 3.3 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.9 MB 12.2 MB/s eta 0:00:01
   --------- ------------------------------ 2.0/8.9 MB 16.1 MB/s eta 0:00:01
   ---------------- ----------------------- 3.7/8.9 MB 21.3 MB/s eta 0:00:01
   ----------------------- ---------------- 5.3/8.9 MB 23.9 MB/s eta 0:00:01
   ------------------------------ --------- 6.8/8.9 MB 25.6 MB/s eta 0:00:01
   -------------------------------------- - 8.6/8.9 MB 28.8 MB/s eta 0:00:01
   ---------------------------------------  8.9/8.9 MB 28.4 MB/s eta 0:00:01
   ---------------------------------------  8.9/8.9 MB 28.4 MB/s eta 0:00:01
   ---------------------------------------- 8.9/8.9 MB 21.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/308.4 kB ? eta -:--:--
   --------------------------------------- 308.4/308.4 kB 19.9 MB/s eta 0:00:00
Note: you m


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Fazreen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
import json, numpy as np, tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

DATA = r"D:\psm2\MediSkinDataset"
IMG=(224,224)
gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_ds = gen.flow_from_directory(f"{DATA}\\test", target_size=IMG, batch_size=32,
                                  class_mode="categorical", shuffle=False)

models = {
    "MobileNetV2": r"D:\psm2\models\disease_mnv2.h5",
    "ResNet50":    r"D:\psm2\models\disease_resnet50.h5",
    "InceptionV3": r"D:\psm2\models\disease_inceptionv3.h5"
}

idx = json.load(open(r"D:\psm2\models\disease_class_indices.json"))
idx_to_cls = {v:k for k,v in idx.items()}

for name, path in models.items():
    print(f"\n=== {name} ===")
    m = load_model(path)
    probs = m.predict(test_ds)
    y_pred = probs.argmax(1); y_true = test_ds.classes
    print(classification_report(y_true, y_pred,
          target_names=[idx_to_cls[i] for i in range(len(idx_to_cls))]))


Found 52 images belonging to 3 classes.

=== MobileNetV2 ===


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
              precision    recall  f1-score   support

   Monkeypox       0.89      0.85      0.87        20
      Others       0.88      0.92      0.90        25
      normal       1.00      1.00      1.00         7

    accuracy                           0.90        52
   macro avg       0.93      0.92      0.92        52
weighted avg       0.90      0.90      0.90        52


=== ResNet50 ===


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step
              precision    recall  f1-score   support

   Monkeypox       0.65      0.85      0.74        20
      Others       0.83      0.76      0.79        25
      normal       1.00      0.43      0.60         7

    accuracy                           0.75        52
   macro avg       0.83      0.68      0.71        52
weighted avg       0.78      0.75      0.75        52


=== InceptionV3 ===


1/2 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000002761E455E40> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step
              precision    recall  f1-score   support

   Monkeypox       0.83      0.95      0.88        20
      Others       0.88      0.84      0.86        25
      normal       1.00      0.71      0.83         7

    accuracy                           0.87        52
   macro avg       0.90      0.83      0.86        52
weighted avg       0.87      0.87      0.86        52



In [9]:
# compare_and_plot.py
import os, json, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
import tensorflow as tf
from tensorflow.keras.models import load_model

# -------- Paths --------
DATA_DIR = r"D:\psm2\MediSkinDataset"
MODELS = {
    "MobileNetV2":  r"D:\psm2\models\disease_mnv2.h5",
    "ResNet50":     r"D:\psm2\models\disease_resnet50.h5",
    "InceptionV3":  r"D:\psm2\models\disease_inceptionv3.h5",
}
CLASS_IDX_JSON = r"D:\psm2\models\disease_class_indices.json"
OUT_DIR = Path(r"D:\psm2\results"); OUT_DIR.mkdir(parents=True, exist_ok=True)

IMG = (224, 224); BATCH = 32

# -------- Data --------
gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_ds = gen.flow_from_directory(
    f"{DATA_DIR}\\test", target_size=IMG, batch_size=BATCH,
    class_mode="categorical", shuffle=False
)

with open(CLASS_IDX_JSON, "r") as f:
    cls2idx = json.load(f)
idx2cls = {v:k for k, v in cls2idx.items()}
class_names = [idx2cls[i] for i in range(len(idx2cls))]

# -------- Helpers --------
def plot_confusion(cm, labels, title, save_path):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    im = ax.imshow(cm)
    ax.set_title(title)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_xticks(range(len(labels))); ax.set_xticklabels(labels, rotation=45, ha="right")
    ax.set_yticks(range(len(labels))); ax.set_yticklabels(labels)
    for i in range(len(labels)):
        for j in range(len(labels)):
            ax.text(j, i, str(cm[i, j]), ha="center", va="center")
    fig.tight_layout()
    fig.savefig(save_path, dpi=200)
    plt.close(fig)

def bar_chart(labels, values, title, ylabel, save_path):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.bar(labels, values)
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_ylim(0, 1.0)
    for i, v in enumerate(values):
        ax.text(i, v + 0.02, f"{v:.2f}", ha="center")
    fig.tight_layout()
    fig.savefig(save_path, dpi=200)
    plt.close(fig)

# -------- Evaluate all models --------
summary_rows = []
per_class_rows = []

for name, model_path in MODELS.items():
    if not os.path.exists(model_path):
        print(f"[WARN] Missing model: {model_path} — skipping.")
        continue

    print(f"\n=== Evaluating {name} ===")
    model = load_model(model_path)

    probs = model.predict(test_ds, verbose=0)
    y_pred = probs.argmax(axis=1)
    y_true = test_ds.classes

    # Metrics
    acc = accuracy_score(y_true, y_pred)
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=list(range(len(class_names))), zero_division=0
    )
    macro_f1 = np.mean(f1)

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(class_names))))

    # Save confusion matrix image
    cm_path = OUT_DIR / f"cm_{name}.png"
    plot_confusion(cm, class_names, f"Confusion Matrix — {name}", cm_path)

    # Save text report
    report_dict = classification_report(
        y_true, y_pred, target_names=class_names, zero_division=0, output_dict=True
    )
    report_txt = OUT_DIR / f"report_{name}.txt"
    with open(report_txt, "w", encoding="utf-8") as f:
        f.write(f"Model: {name}\n\n")
        f.write(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))
        f.write("\nConfusion Matrix:\n")
        f.write(np.array2string(cm))

    # Append to summary
    summary_rows.append((name, acc, macro_f1))

    # Per-class rows
    for i, cls in enumerate(class_names):
        per_class_rows.append((name, cls, precision[i], recall[i], f1[i], int(support[i])))

# -------- Save CSVs --------
import csv
summary_csv = OUT_DIR / "summary_overall.csv"
with open(summary_csv, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f); w.writerow(["Model", "Accuracy", "Macro_F1"])
    for row in summary_rows:
        w.writerow(row)

perclass_csv = OUT_DIR / "summary_per_class.csv"
with open(perclass_csv, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f); w.writerow(["Model", "Class", "Precision", "Recall", "F1", "Support"])
    for row in per_class_rows:
        w.writerow(row)

# -------- Comparison plots --------
# Overall Accuracy
labels = [r[0] for r in summary_rows]
accs   = [r[1] for r in summary_rows]
bar_chart(labels, accs, "Overall Accuracy Comparison", "Accuracy", OUT_DIR / "accuracy_comparison.png")

# Overall Macro-F1
mfs = [r[2] for r in summary_rows]
bar_chart(labels, mfs, "Macro-F1 Comparison", "Macro-F1", OUT_DIR / "macroF1_comparison.png")

print("\n✅ Done. Results saved to:", OUT_DIR)
print(" - Confusion matrices: cm_*.png")
print(" - Per-model reports: report_*.txt")
print(" - CSVs: summary_overall.csv, summary_per_class.csv")
print(" - Charts: accuracy_comparison.png, macroF1_comparison.png")


Found 52 images belonging to 3 classes.

=== Evaluating MobileNetV2 ===


C:\Users\Fazreen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



=== Evaluating ResNet50 ===



=== Evaluating InceptionV3 ===



✅ Done. Results saved to: D:\psm2\results
 - Confusion matrices: cm_*.png
 - Per-model reports: report_*.txt
 - CSVs: summary_overall.csv, summary_per_class.csv
 - Charts: accuracy_comparison.png, macroF1_comparison.png
